In [1]:
import hermes
from hermes.analysis import LOSAnalysis
from hermes.constellations.Telesat import Telesat_00053
from hermes.objects import Satellite, SatGroup, Earth, Constellation
from hermes.scenario import Scenario
from hermes.simulation import Simulation
from hermes.util import hex2rgb

from pyreport import PlotUtil

from astropy import units as u, time
import numpy as np

%matplotlib

gzip was not found on your system! You should solve this issue for astroquery.eso to be at its best!
On POSIX system: make sure gzip is installed and in your path!On Windows: same for 7-zip (http://www.7-zip.org)!
Using matplotlib backend: Qt5Agg


C:\git\hermes-optimization\cenv\lib\site-packages\astropy\units\decorators.py:222: UserWarning: Wrapping true anomaly to -π <= nu < π
  return_ = wrapped_function(*func_args, **func_kwargs)


In [2]:
from use_cases import IoTM2M_mission, O3b_target

case = IoTM2M_mission
target = O3b_target

import util
util.print_targets(case)

import pickle
pickle.dump((case, target), open('raw/iot_settings.pkl', 'wb'))

Targets per orbit
T_target: 20.00 Gb (min: 20.00 Gb, max: -0.00 Gb)
L_target: 60.00 s
E_target: 16.82 kJ (max: -0.00 kJ)
P_target: 0.00 s


C:\git\hermes-optimization\cenv\lib\site-packages\astropy\units\decorators.py:222: UserWarning: Wrapping true anomaly to -π <= nu < π
  return_ = wrapped_function(*func_args, **func_kwargs)


In [3]:
T_sim = case['T_sim_s']

start = time.Time('2019-09-01 10:00:00.000', scale='tt')        # Start time of simulation
stop = start + T_sim * u.s # Stop time of simulation
step = 1 * u.s

scenario = Scenario(start, stop, step, hermes.objects.Earth, name='IoT_O3b')
scenario.add_satellite(case['satellite'])
scenario.add_satellite(target['constellation'])
scenario.add_analysis(LOSAnalysis(scenario, case['satellite'], target['constellation']))

# Initialise the scenario
scenario.initialise()

simulation = Simulation(scenario, show_3d=False)
simulation.run()

Initializing attractor
Initializing 43 satellites...
Initializing analysis 1 of 1...


Simulation progress: 17404it [00:29, 598.17it/s]                            


In [4]:
import shutil

# Close store
scenario.state.analyses[0].store.close()

# move output file to /raw
shutil.move(scenario.state.analyses[0].hdf_name, "raw/" + scenario.state.analyses[0].hdf_name)


'raw/2021-04-25_14-46-56_IoT_O3b_LOSAnalysis_sat_to_node.h5'

In [5]:
columns = ['strand_name', 'tof',
           'r_a_x', 'r_a_y', 'r_a_z',
           'v_a_x', 'v_a_y', 'v_a_z',
           'r_b_x', 'r_b_y', 'r_b_z']

file_path = "raw/" + scenario.state.analyses[0].hdf_name

row_limit = -1

import pandas as pd

hdf_store = pd.HDFStore(file_path)
instances_df = hdf_store.select('contact_instances', 'columns = %s' % str(columns), stop = row_limit)

# Sort by pass id
instances_df = instances_df.sort_index(0)

from hermes.postprocessing import *

# Add range column d
instances_df = add_range(instances_df)

# Add sff
instances_df = add_sff(instances_df)

instances_df.to_hdf('raw/iot_o3b_post_processed.h5', 'contact_instances', format='table')

hdf_store.close()



C:\git\hermes-simulator\hermes\geometry.py:278: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, A))
  v[i, 0] = np.dot(R_x[i, :], u[i, :])
C:\git\hermes-simulator\hermes\geometry.py:279: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, C), array(float64, 1d, A))
  v[i, 1] = np.dot(R_y[i, :], u[i, :])
C:\git\hermes-simulator\hermes\geometry.py:280: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, A))
  v[i, 2] = np.dot(R_z[i, :], u[i, :])
